In [11]:
using DataFrames
using GenX
using CSV
using YAML

In [65]:
function reconstruction(case::AbstractString,DF::DataFrame)
    settings_path = GenX.get_settings_path(case)
    
    # Read Period map file Period_map.csv
    Period_map = CSV.read(joinpath(case,"TDR_results/Period_map.csv"),DataFrame)
    
    # Read time domain reduction settings file time_domain_reduction_settings.yml
    myTDRsetup = YAML.load(open(joinpath(settings_path,
        "time_domain_reduction_settings.yml")))
    
    # Define Timesteps per Representative Period and Weight Total
    TimestepsPerRepPeriod = myTDRsetup["TimestepsPerRepPeriod"]
    WeightTotal = myTDRsetup["WeightTotal"]
    
    # Calculate the number of total periods the original time series was split into (will usually be 52)
    numPeriods = floor(Int,WeightTotal/TimestepsPerRepPeriod)
    
    # Get the names of the input DataFrame
    DFnames = names(DF)
    
    # Initialize an array to add the reconstructed data to
    recon = ["t$t" for t in 1:TimestepsPerRepPeriod*numPeriods]
    
    # Find the index of the row with the first time step
    t1 = findfirst(x -> x == "t1",DF[!,1])
    
    # Reconstruction of all hours of the year from TDR
    for j in range(2,ncol(DF))
        col = DF[t1:end,j]
        col_name = DFnames[j]
        recon_col = []
        for i in range(1,numPeriods)
            index = Period_map[i,"Rep_Period_Index"]
            recon_temp = col[(TimestepsPerRepPeriod*index-(TimestepsPerRepPeriod-1)):(TimestepsPerRepPeriod*index)]
            recon_col = [recon_col; recon_temp]
        end
        recon = [recon recon_col]
    end
    reconDF = DataFrame(recon, DFnames)
    
    # Insert rows that were above "t1" in the original DataFrame (e.g. "Zone" and "AnnualSum") if present
    for i in range(1,t1-1)
        insert!(reconDF,i,DF[i,1:end])
    end
    
    # Repeat the last rows of the year to fill in the gap (should be 24 hours for non-leap year)
    end_diff = WeightTotal - nrow(reconDF) + 1
    new_rows = reconDF[(nrow(reconDF)-end_diff):nrow(reconDF),2:end]
    new_rows[!,"Resource"] = ["t$t" for t in (WeightTotal-end_diff):WeightTotal] 
    
    reconDF = [reconDF; new_rows]
    
    return reconDF
end

reconstruction (generic function with 1 method)

In [67]:
power_recon = reconstruction(case,power)

Row,Resource,MA_natural_gas_combined_cycle,CT_natural_gas_combined_cycle,ME_natural_gas_combined_cycle,MA_solar_pv,CT_onshore_wind,CT_solar_pv,ME_onshore_wind,MA_battery,CT_battery,ME_battery,Total
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Zone,1.0,2.0,3.0,1.0,2.0,2.0,3.0,1.0,2.0,3.0,0.0
2,AnnualSum,1.05862e7,3.4267e6,8.85336e5,3.49456e7,3.54989e7,1.42853e7,2.33054e7,9.73799e6,1.3938e7,3.7731e6,1.50383e8
3,t1,-0.0,-0.0,-0.0,-0.0,8577.12,-0.0,5175.24,854.11,2309.3,650.686,17566.5
4,t2,-0.0,-0.0,-0.0,-0.0,9379.44,-0.0,4216.19,1720.89,1847.14,0.0,17163.7
5,t3,-0.0,-0.0,-0.0,-0.0,10446.9,-0.0,3868.49,482.61,1267.93,1253.56,17319.5
6,t4,-0.0,-0.0,-0.0,-0.0,9742.74,-0.0,2314.3,2382.96,1597.49,0.0,16037.5
7,t5,-0.0,-0.0,-0.0,-0.0,5745.32,-0.0,1057.74,2370.46,3592.7,626.779,13393.0
8,t6,-0.0,-0.0,-0.0,-0.0,3680.74,-0.0,4152.22,3682.61,1848.39,626.779,13990.7
9,t7,-0.0,-0.0,-0.0,-0.0,4415.34,-0.0,1109.74,2562.96,4312.7,626.779,13027.5


In [45]:
case = joinpath("example_systems/1_three_zones");
power =  CSV.read(joinpath(case,"results/power.csv"),DataFrame,missingstring="NA")

Row,Resource,MA_natural_gas_combined_cycle,CT_natural_gas_combined_cycle,ME_natural_gas_combined_cycle,MA_solar_pv,CT_onshore_wind,CT_solar_pv,ME_onshore_wind,MA_battery,CT_battery,ME_battery,Total
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Zone,1.0,2.0,3.0,1.0,2.0,2.0,3.0,1.0,2.0,3.0,0.0
2,AnnualSum,1.05862e7,3.4267e6,8.85336e5,3.49456e7,3.54989e7,1.42853e7,2.33054e7,9.73799e6,1.3938e7,3.7731e6,1.50383e8
3,t1,-0.0,-0.0,-0.0,-0.0,8577.12,-0.0,5175.24,854.11,2309.3,650.686,17566.5
4,t2,-0.0,-0.0,-0.0,-0.0,9379.44,-0.0,4216.19,1720.89,1847.14,0.0,17163.7
5,t3,-0.0,-0.0,-0.0,-0.0,10446.9,-0.0,3868.49,482.61,1267.93,1253.56,17319.5
6,t4,-0.0,-0.0,-0.0,-0.0,9742.74,-0.0,2314.3,2382.96,1597.49,0.0,16037.5
7,t5,-0.0,-0.0,-0.0,-0.0,5745.32,-0.0,1057.74,2370.46,3592.7,626.779,13393.0
8,t6,-0.0,-0.0,-0.0,-0.0,3680.74,-0.0,4152.22,3682.61,1848.39,626.779,13990.7
9,t7,-0.0,-0.0,-0.0,-0.0,4415.34,-0.0,1109.74,2562.96,4312.7,626.779,13027.5


In [46]:
power_recon = reconstruction(case,power)

["t8736", "t8737", "t8738", "t8739", "t8740", "t8741", "t8742", "t8743", "t8744", "t8745", "t8746", "t8747", "t8748", "t8749", "t8750", "t8751", "t8752", "t8753", "t8754", "t8755", "t8756", "t8757", "t8758", "t8759", "t8760"]


Row,Resource,MA_natural_gas_combined_cycle,CT_natural_gas_combined_cycle,ME_natural_gas_combined_cycle,MA_solar_pv,CT_onshore_wind,CT_solar_pv,ME_onshore_wind,MA_battery,CT_battery,ME_battery,Total
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Zone,1.0,2.0,3.0,1.0,2.0,2.0,3.0,1.0,2.0,3.0,0.0
2,AnnualSum,1.05862e7,3.4267e6,8.85336e5,3.49456e7,3.54989e7,1.42853e7,2.33054e7,9.73799e6,1.3938e7,3.7731e6,1.50383e8
3,t1,-0.0,-0.0,-0.0,-0.0,8577.12,-0.0,5175.24,854.11,2309.3,650.686,17566.5
4,t2,-0.0,-0.0,-0.0,-0.0,9379.44,-0.0,4216.19,1720.89,1847.14,0.0,17163.7
5,t3,-0.0,-0.0,-0.0,-0.0,10446.9,-0.0,3868.49,482.61,1267.93,1253.56,17319.5
6,t4,-0.0,-0.0,-0.0,-0.0,9742.74,-0.0,2314.3,2382.96,1597.49,0.0,16037.5
7,t5,-0.0,-0.0,-0.0,-0.0,5745.32,-0.0,1057.74,2370.46,3592.7,626.779,13393.0
8,t6,-0.0,-0.0,-0.0,-0.0,3680.74,-0.0,4152.22,3682.61,1848.39,626.779,13990.7
9,t7,-0.0,-0.0,-0.0,-0.0,4415.34,-0.0,1109.74,2562.96,4312.7,626.779,13027.5


In [18]:
println(365*24)
println(52*168)
println(365/7)

8760
8736
52.142857142857146


In [19]:
print(21481/5/24)

179.00833333333333